# Introduction
In this chapter, we will learn
- Enums: allow you to define a type by enumerating its possible variants
- `Option` enum: a very useful enum which expresses that a value can be either something or nothing.
- Patterns matching with `match`
- Enum control flow using `if let`

# Defining an Enum

<span style="color:lightgreen">***Where structs give you a way of grouping together related fields and data, like a `Rectangle` with its `width` and `height`, enums give you a way of saying a value is one of a possible set of values.***</span>


Example: Let's make a small program to represent IP addresses. Any IP address can be either a version 4 or a version 6 address, but not both at the same time. Both versions are still fundamentally IP addresses and should be treated as the same type when the code is handling situations that apply to any kind of IP address.

In [6]:
#[derive(Debug)]
enum IpAddrKind {
    V4,
    V6,
}

fn main() {
    let four = IpAddrKind::V4;
    let six = IpAddrKind::V6;

    route(IpAddrKind::V4);
    route(IpAddrKind::V6);
}

fn route(ip_kind: IpAddrKind) {
    println!("We are routing using IP {:?}", ip_kind);
}

main()

We are routing using IP V4
We are routing using IP V6


()

We can put data directly into each enum variant.

In [17]:
#[derive(Debug)]
enum IpAddr {
    V4(String),
    V6(String),
}

let home = IpAddr::V4(String::from("127.0.0.1"));
let loopback = IpAddr::V6(String::from("::1"));
println!("{:?}", home);
println!("{:?}", loopback);

V4("127.0.0.1")
V6("::1")


<span style="color:lightgreen">*There’s another advantage to using an enum rather than a struct: each variant can have different types and amounts of associated data.*</span>

In [19]:
#[derive(Debug)]
enum IpAddr {
    V4(u8, u8, u8, u8),
    V6(String),
}

let home = IpAddr::V4(127, 0, 0, 100);

let loopback = IpAddr::V6(String::from("::2"));
println!("{:?}", home);
println!("{:?}", loopback);

V4(127, 0, 0, 100)
V6("::2")


<span style="color:skyblue">***You can put any kind of data inside an enum variant: strings, numeric types, or structs, for example. You can even include another enum!***</span>

Another example to illustrate enum vs structs

In [20]:
enum Message {
    Quit,  // Quit has no data associated with it at all.
    Move { x: i32, y: i32 },  // Move has named fields, like a struct does.
    Write(String),  // Write includes a single String.
    ChangeColor(i32, i32, i32),  // ChangeColor includes three i32 values.
}

// we can define structs to hold similar data
struct QuitMessage; // unit struct
struct MoveMessage { x: i32, y: i32, }  // named fields struct
struct WriteMessage(String); // tuple struct
struct ChangeColorMessage(i32, i32, i32); // tuple struct


### Methods on Enums Using `impl`

In [21]:
impl Message {
    fn call(&self) {
        // method body would be defined here
    }
}

let m = Message::Write(String::from("hello"));
m.call();


## The `Option` Enum and Its Advantages Over Null Values

The `Option` enum belongs to the standard library and <span style="color:lightgreen">*encodes the very common scenario in which a value could be something or it could be nothing.*</span> For example, if you request the first item in a non-empty list, you would get a value. If you request the first item in an empty list, you would get nothing.

<span style="color:lightgreen">*`Null` is a value that means there is no value there. Rust doesn’t have the null feature that many other languages have, since it has led to innumerable errors, vulnerabilities, and system crashes, which have probably caused a billion dollars of pain and damage in the last forty years.*</span>

<span style="color:orange">*The problem with null values is that if you try to use a null value as a not-null value, you’ll get an error of some kind. Because this null or not-null property is pervasive, it’s extremely easy to make this kind of error.*</span>

Rust does not have Null, but it has the `Option<T>` enum in the prelude, as also its variants (`Some` and `None`). You can use them directly without the `Option::` prefix
```rust
enum Option<T> {
    None,
    Some(T),
}
```

In [4]:
let some_number: Option<i32> = Some(5);
let some_char: Option<char> = Some('e');

let absent_number: Option<i32> = None;

println!("{:?}", some_char);
println!("{:?}", some_number);
println!("{:?}", absent_number);

Some('e')
Some(5)
None


### Why `Option<T>` is better than `Null`?

<span style="color:lightgreen">***When we have a `Some` value, we know that a value is present and the value is held within the `Some`. When we have a `None` value, in some sense it means the same thing as null: we don’t have a valid value. So why is having `Option<T>` any better than having null? In short, because `Option<T>` and `T` (where `T` can be any type) are different types, the compiler won’t let us use an `Option<T>` value as if it were definitely a valid value.***</span>

The code below won't work:

In [5]:
let x: i8 = 5;
let y: Option<i8> = Some(5);

let sum = x + y;


Error: cannot add `Option<i8>` to `i8`

When we have a value of a type like `i8` in Rust, the compiler will ensure that we always have a valid value. We can proceed confidently without having to check for null before using that value. When we have an `Option<T>`, we have to convert an `Option<T>` to a `T` before you can perform `T` operations with it. Generally, this helps catch one of the most common issues with null: assuming that something isn’t null when it actually is. <span style="color:lightgreen">*In order to have a value that can possibly be null, you must explicitly opt in by making the type of that value `Option<T>`. Then, when you use that value, you are required to explicitly handle the case when the value is null.*</span>

# The `match` Control Flow Construct
<span style="color:lightgreen">***Rust has an extremely powerful control flow construct called `match` that allows you to compare a value against a series of patterns and then execute code based on which pattern matches***</span>. Patterns can be made up of literal values, variable names, wildcards, and many other things;

In [17]:
enum Coin {
    Penny,
    Nickel,
    Dime,
    Quarter,
}

fn value_in_cents(coin: Coin) -> u8 {
    match coin {
        Coin::Penny => {  
            println!("Lucky penny!");
            1  // The code associated with each arm is an expression
        }
        Coin::Nickel => 5,
        Coin::Dime => 10,
        Coin::Quarter => 25,
    }
}

let penny = Coin::Penny; 
let penny_value = value_in_cents(penny);
dbg!(penny_value);

Lucky penny!


[src/lib.rs:124] penny_value = 1


## Patterns That Bind to Values
Another useful feature of match arms is that they can bind to the parts of the values that match the pattern. This is how we can extract values out of enum variants.

In [18]:
#[derive(Debug)]
enum UsState {
    Alabama,
    Alaska,
    // --snip--
}

enum Coin {
    Penny,
    Nickel,
    Dime,
    Quarter(UsState), // When a Coin::Quarter matches, the state variable will bind to the value of that quarter’s state
}

fn value_in_cents(coin: Coin) -> u8 {
    match coin {
        Coin::Penny => 1,
        Coin::Nickel => 5,
        Coin::Dime => 10,
        Coin::Quarter(state) => {
            println!("State quarter from {:?}!", state);
            25
        }
    }
}

fn main() {
    value_in_cents(Coin::Quarter(UsState::Alaska));
}

main()

State quarter from Alaska!


()

## Matching with `Option<T>`

In [24]:
fn plus_one(x: Option<i32>) -> Option<i32> {
    match x {
        None => None,
        Some(i) => Some(i + 1),
    }
}

let six = plus_one(Some(5));
let none = plus_one(None);
dbg!(six);
dbg!(none);

[src/lib.rs:146] six = Some(
    6,
)
[src/lib.rs:147] none = None


> **Note: Using `match`**
> 
> You’ll see this pattern a lot in Rust code: <span style="color:skyblue">***`match` against an `enum`, bind a variable to the data inside, and then execute code based on it***</span>. It’s a bit tricky at first, but once you get used to it, you’ll wish you had it in all languages. It’s consistently a user favorite.

## Matches Are Exhaustive

<span style="color:orange">***March arms’ patterns must cover all possibilities***</span>

The following code fails as it does not cover all possibilities:

In [26]:
fn plus_one(x: Option<i32>) -> Option<i32> {
    match x {
        Some(i) => Some(i + 1),
        // we forgot the None case here
    }
}

Error: non-exhaustive patterns: `None` not covered

## Catch-all Patterns and the `_` Placeholder

- We use `other` in `match` to cover every other possible value

In [31]:
fn main() {
    let dice_roll = 9;
    match dice_roll {
        3 => add_fancy_hat(),
        7 => remove_fancy_hat(),
        other => move_player(other),
    }

    fn add_fancy_hat() {}
    fn remove_fancy_hat() {}
    fn move_player(dr: u8) {println!("moving player since dice roll is {}", dr)}
}
main()

moving player since dice roll is 9


()

- We use as `_` to match any value and don't bind to that value

In [35]:
let dice_roll = 9;
match dice_roll {
    3 => add_fancy_hat(),
    7 => remove_fancy_hat(),
    _ => reroll(),  // pass if the value does not match
}

fn add_fancy_hat() {}
fn remove_fancy_hat() {}
fn reroll() {}

# Concise Control Flow with `if let`

<span style="color:lightgreen">*`if let` is a syntax sugar for a `match` that runs code when the value matches one pattern and then ignores all other values.*</span>

Intead of consistently writing the `_ => ()` and `match` as in

In [37]:
let config_max = Some(3u8);
match config_max {
    Some(max) => println!("The maximum is configured to be {}", max),
    _ => (),
}

The maximum is configured to be 3


()

we can use `if let` to achieve the same thing:

In [38]:
let config_max = Some(3u8);
if let Some(max) = config_max {
    println!("The maximum is configured to be {}", max);
}

The maximum is configured to be 3


()

## `else` in `if let`

We can include an `else` with an `if let`. The block of code that goes with the else is the same as the block of code that would go with the `_` case in the `match` expression

In [50]:
#[derive(Debug)]
enum UsState {
    Alabama,
    Alaska,
    // --snip--
}

enum Coin {
    Penny,
    Nickel,
    Dime,
    Quarter(UsState),
}

fn using_match() {
    let coin = Coin::Quarter(UsState::Alaska);
    println!("-- using match: ");
    let mut count = 0;
    match coin {
        Coin::Quarter(state) => println!("State quarter from {:?}!", state),
        _ => count += 1,
    }
}

fn using_if_let() {
    let coin = Coin::Quarter(UsState::Alaska);
    // using if let
    println!("-- using if let: ");
    let mut count = 0;
    if let Coin::Quarter(state) = coin {
        println!("State quarter from {:?}!", state);
    } else {
        count += 1;
    }
}

using_match();
using_if_let();

-- using match: 
State quarter from Alaska!
-- using if let: 
State quarter from Alaska!
